# SGLang Frontend Language

SGLang frontend language can be used to define simple and easy prompts in a convenient, structured way.

## Launch A Server

Launch the server in your terminal and wait for it to initialize.

In [1]:
import requests
import os

from sglang import assistant_begin, assistant_end
from sglang import assistant, function, gen, system, user
from sglang import image
from sglang import RuntimeEndpoint, set_default_backend
from sglang.srt.utils import load_image
from sglang.test.test_utils import is_in_ci
from sglang.utils import print_highlight, terminate_process, wait_for_server

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path Qwen/Qwen2.5-7B-Instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")
print(f"Server started on http://localhost:{port}")

[2025-02-26 23:39:27] server_args=ServerArgs(model_path='Qwen/Qwen2.5-7B-Instruct', tokenizer_path='Qwen/Qwen2.5-7B-Instruct', tokenizer_mode='auto', load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization_param_path=None, quantization=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen2.5-7B-Instruct', chat_template=None, is_embedding=False, revision=None, skip_tokenizer_init=False, host='0.0.0.0', port=31635, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, cpu_offload_gb=0, prefill_only_one_req=False, tp_size=1, stream_interval=1, stream_output=False, random_seed=1049487264, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, log_level='info', log_level_http=None, log_requests=False, show_time_cost=False, enable_metrics=False, dec

[2025-02-26 23:39:47 TP0] Init torch distributed begin.
[2025-02-26 23:39:47 TP0] Load weight begin. avail mem=62.69 GB


[2025-02-26 23:39:47 TP0] The following error message 'operation scheduled before its operands' can be ignored.


[2025-02-26 23:39:48 TP0] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

[2025-02-26 23:39:52 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768


[2025-02-26 23:39:52] INFO:     Started server process [1859622]
[2025-02-26 23:39:52] INFO:     Waiting for application startup.
[2025-02-26 23:39:52] INFO:     Application startup complete.
[2025-02-26 23:39:52] INFO:     Uvicorn running on http://0.0.0.0:31635 (Press CTRL+C to quit)


[2025-02-26 23:39:53] INFO:     127.0.0.1:45062 - "GET /v1/models HTTP/1.1" 200 OK


[2025-02-26 23:39:53] INFO:     127.0.0.1:45068 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-02-26 23:39:53 TP0] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


Server started on http://localhost:31635


Set the default backend. Note: Besides the local server, you may use also `OpenAI` or other API endpoints.

In [2]:
set_default_backend(RuntimeEndpoint(f"http://localhost:{port}"))

[2025-02-26 23:39:58] INFO:     127.0.0.1:45080 - "GET /get_model_info HTTP/1.1" 200 OK


## Basic Usage

The most simple way of using SGLang frontend language is a simple question answer dialog between a user and an assistant.

In [3]:
@function
def basic_qa(s, question):
    s += system(f"You are a helpful assistant than can answer questions.")
    s += user(question)
    s += assistant(gen("answer", max_tokens=512))

In [4]:
state = basic_qa("List 3 countries and their capitals.")
print_highlight(state["answer"])

[2025-02-26 23:39:59 TP0] Prefill batch. #new-seq: 1, #new-token: 31, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 1, #queue-req: 0


[2025-02-26 23:39:59] INFO:     127.0.0.1:45078 - "POST /generate HTTP/1.1" 200 OK
[2025-02-26 23:39:59] The server is fired up and ready to roll!


[2025-02-26 23:39:59] INFO:     127.0.0.1:45094 - "POST /generate HTTP/1.1" 200 OK


## Multi-turn Dialog

SGLang frontend language can also be used to define multi-turn dialogs.

In [5]:
@function
def multi_turn_qa(s):
    s += system(f"You are a helpful assistant than can answer questions.")
    s += user("Please give me a list of 3 countries and their capitals.")
    s += assistant(gen("first_answer", max_tokens=512))
    s += user("Please give me another list of 3 countries and their capitals.")
    s += assistant(gen("second_answer", max_tokens=512))
    return s


state = multi_turn_qa()
print_highlight(state["first_answer"])
print_highlight(state["second_answer"])

[2025-02-26 23:40:00 TP0] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 18, cache hit rate: 24.66%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-02-26 23:40:00 TP0] Decode batch. #running-req: 1, #token: 48, token usage: 0.00, gen throughput (token/s): 5.99, #queue-req: 0


[2025-02-26 23:40:00] INFO:     127.0.0.1:45108 - "POST /generate HTTP/1.1" 200 OK


[2025-02-26 23:40:00 TP0] Prefill batch. #new-seq: 1, #new-token: 23, #cached-token: 70, cache hit rate: 53.01%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-02-26 23:40:00 TP0] Decode batch. #running-req: 1, #token: 110, token usage: 0.01, gen throughput (token/s): 57.73, #queue-req: 0


[2025-02-26 23:40:01] INFO:     127.0.0.1:45112 - "POST /generate HTTP/1.1" 200 OK


## Control flow

You may use any Python code within the function to define more complex control flows.

In [6]:
@function
def tool_use(s, question):
    s += assistant(
        "To answer this question: "
        + question
        + ". I need to use a "
        + gen("tool", choices=["calculator", "search engine"])
        + ". "
    )

    if s["tool"] == "calculator":
        s += assistant("The math expression is: " + gen("expression"))
    elif s["tool"] == "search engine":
        s += assistant("The key word to search is: " + gen("word"))


state = tool_use("What is 2 * 2?")
print_highlight(state["tool"])
print_highlight(state["expression"])

[2025-02-26 23:40:01 TP0] Prefill batch. #new-seq: 1, #new-token: 25, #cached-token: 8, cache hit rate: 48.24%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-02-26 23:40:01] INFO:     127.0.0.1:40912 - "POST /generate HTTP/1.1" 200 OK
[2025-02-26 23:40:01 TP0] Prefill batch. #new-seq: 1, #new-token: 2, #cached-token: 31, cache hit rate: 54.74%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-02-26 23:40:01 TP0] Prefill batch. #new-seq: 1, #new-token: 3, #cached-token: 31, cache hit rate: 59.40%, token usage: 0.00, #running-req: 1, #queue-req: 0
[2025-02-26 23:40:01] INFO:     127.0.0.1:40922 - "POST /generate HTTP/1.1" 200 OK


[2025-02-26 23:40:01 TP0] Prefill batch. #new-seq: 1, #new-token: 13, #cached-token: 33, cache hit rate: 61.22%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-02-26 23:40:01 TP0] Decode batch. #running-req: 1, #token: 68, token usage: 0.00, gen throughput (token/s): 59.79, #queue-req: 0


[2025-02-26 23:40:02 TP0] Decode batch. #running-req: 1, #token: 108, token usage: 0.01, gen throughput (token/s): 60.68, #queue-req: 0


[2025-02-26 23:40:02] INFO:     127.0.0.1:40936 - "POST /generate HTTP/1.1" 200 OK


## Parallelism

Use `fork` to launch parallel prompts. Because `sgl.gen` is non-blocking, the for loop below issues two generation calls in parallel.

In [7]:
@function
def tip_suggestion(s):
    s += assistant(
        "Here are two tips for staying healthy: "
        "1. Balanced Diet. 2. Regular Exercise.\n\n"
    )

    forks = s.fork(2)
    for i, f in enumerate(forks):
        f += assistant(
            f"Now, expand tip {i+1} into a paragraph:\n"
            + gen("detailed_tip", max_tokens=256, stop="\n\n")
        )

    s += assistant("Tip 1:" + forks[0]["detailed_tip"] + "\n")
    s += assistant("Tip 2:" + forks[1]["detailed_tip"] + "\n")
    s += assistant(
        "To summarize the above two tips, I can say:\n" + gen("summary", max_tokens=512)
    )


state = tip_suggestion()
print_highlight(state["summary"])

[2025-02-26 23:40:02 TP0] Prefill batch. #new-seq: 1, #new-token: 35, #cached-token: 14, cache hit rate: 56.79%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-02-26 23:40:02 TP0] Prefill batch. #new-seq: 1, #new-token: 35, #cached-token: 14, cache hit rate: 53.41%, token usage: 0.00, #running-req: 1, #queue-req: 0


[2025-02-26 23:40:02 TP0] Decode batch. #running-req: 2, #token: 104, token usage: 0.01, gen throughput (token/s): 89.61, #queue-req: 0


[2025-02-26 23:40:03 TP0] Decode batch. #running-req: 2, #token: 184, token usage: 0.01, gen throughput (token/s): 119.70, #queue-req: 0


[2025-02-26 23:40:04 TP0] Decode batch. #running-req: 2, #token: 264, token usage: 0.01, gen throughput (token/s): 122.52, #queue-req: 0


[2025-02-26 23:40:04] INFO:     127.0.0.1:40938 - "POST /generate HTTP/1.1" 200 OK


[2025-02-26 23:40:04] INFO:     127.0.0.1:40952 - "POST /generate HTTP/1.1" 200 OK
[2025-02-26 23:40:04 TP0] Prefill batch. #new-seq: 1, #new-token: 300, #cached-token: 39, cache hit rate: 34.45%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-02-26 23:40:04 TP0] Decode batch. #running-req: 1, #token: 341, token usage: 0.02, gen throughput (token/s): 96.99, #queue-req: 0


[2025-02-26 23:40:05 TP0] Decode batch. #running-req: 1, #token: 381, token usage: 0.02, gen throughput (token/s): 60.76, #queue-req: 0


[2025-02-26 23:40:06 TP0] Decode batch. #running-req: 1, #token: 421, token usage: 0.02, gen throughput (token/s): 59.29, #queue-req: 0


[2025-02-26 23:40:06 TP0] Decode batch. #running-req: 1, #token: 461, token usage: 0.02, gen throughput (token/s): 59.71, #queue-req: 0
[2025-02-26 23:40:07] INFO:     127.0.0.1:40958 - "POST /generate HTTP/1.1" 200 OK


## Constrained Decoding

Use `regex` to specify a regular expression as a decoding constraint. This is only supported for local models.

In [8]:
@function
def regular_expression_gen(s):
    s += user("What is the IP address of the Google DNS servers?")
    s += assistant(
        gen(
            "answer",
            temperature=0,
            regex=r"((25[0-5]|2[0-4]\d|[01]?\d\d?).){3}(25[0-5]|2[0-4]\d|[01]?\d\d?)",
        )
    )


state = regular_expression_gen()
print_highlight(state["answer"])

Compiling FSM index for all state transitions:   0%|          | 0/69 [00:00<?, ?it/s]

Compiling FSM index for all state transitions:   6%|▌         | 4/69 [00:00<00:05, 11.16it/s]

Compiling FSM index for all state transitions:  16%|█▌        | 11/69 [00:00<00:02, 20.40it/s]

Compiling FSM index for all state transitions:  25%|██▍       | 17/69 [00:00<00:02, 24.79it/s]

Compiling FSM index for all state transitions:  35%|███▍      | 24/69 [00:01<00:01, 25.77it/s]

...

Compiling FSM index for all state transitions:  81%|████████  | 56/69 [00:02<00:00, 23.46it/s]

Compiling FSM index for all state transitions:  91%|█████████▏| 63/69 [00:02<00:00, 25.27it/s]

Compiling FSM index for all state transitions: 100%|██████████| 69/69 [00:02<00:00, 23.57it/s]


[2025-02-26 23:40:20 TP0] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 12, cache hit rate: 34.66%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-02-26 23:40:21] INFO:     127.0.0.1:40970 - "POST /generate HTTP/1.1" 200 OK


Use `regex` to define a `JSON` decoding schema.

In [9]:
character_regex = (
    r"""\{\n"""
    + r"""    "name": "[\w\d\s]{1,16}",\n"""
    + r"""    "house": "(Gryffindor|Slytherin|Ravenclaw|Hufflepuff)",\n"""
    + r"""    "blood status": "(Pure-blood|Half-blood|Muggle-born)",\n"""
    + r"""    "occupation": "(student|teacher|auror|ministry of magic|death eater|order of the phoenix)",\n"""
    + r"""    "wand": \{\n"""
    + r"""        "wood": "[\w\d\s]{1,16}",\n"""
    + r"""        "core": "[\w\d\s]{1,16}",\n"""
    + r"""        "length": [0-9]{1,2}\.[0-9]{0,2}\n"""
    + r"""    \},\n"""
    + r"""    "alive": "(Alive|Deceased)",\n"""
    + r"""    "patronus": "[\w\d\s]{1,16}",\n"""
    + r"""    "bogart": "[\w\d\s]{1,16}"\n"""
    + r"""\}"""
)


@function
def character_gen(s, name):
    s += user(
        f"{name} is a character in Harry Potter. Please fill in the following information about this character."
    )
    s += assistant(gen("json_output", max_tokens=256, regex=character_regex))


state = character_gen("Harry Potter")
print_highlight(state["json_output"])

Compiling FSM index for all state transitions:   0%|          | 0/431 [00:00<?, ?it/s]

Compiling FSM index for all state transitions:   1%|          | 5/431 [00:00<00:27, 15.57it/s]

Compiling FSM index for all state transitions:   3%|▎         | 13/431 [00:00<00:15, 26.39it/s]

Compiling FSM index for all state transitions:   4%|▎         | 16/431 [00:00<00:17, 23.94it/s]

Compiling FSM index for all state transitions:   5%|▌         | 23/431 [00:01<00:17, 22.79it/s]

...

[2025-02-26 23:40:39 TP0] Prefill batch. #new-seq: 1, #new-token: 24, #cached-token: 14, cache hit rate: 34.76%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-02-26 23:40:39 TP0] Decode batch. #running-req: 1, #token: 60, token usage: 0.00, gen throughput (token/s): 1.22, #queue-req: 0


[2025-02-26 23:40:40 TP0] Decode batch. #running-req: 1, #token: 100, token usage: 0.00, gen throughput (token/s): 83.99, #queue-req: 0


[2025-02-26 23:40:40 TP0] Decode batch. #running-req: 1, #token: 140, token usage: 0.01, gen throughput (token/s): 82.31, #queue-req: 0


[2025-02-26 23:40:41] INFO:     127.0.0.1:46552 - "POST /generate HTTP/1.1" 200 OK


## Batching 

Use `run_batch` to run a batch of prompts.

In [10]:
@function
def text_qa(s, question):
    s += user(question)
    s += assistant(gen("answer", stop="\n"))


states = text_qa.run_batch(
    [
        {"question": "What is the capital of the United Kingdom?"},
        {"question": "What is the capital of France?"},
        {"question": "What is the capital of Japan?"},
    ],
    progress_bar=True,
)

for i, state in enumerate(states):
    print_highlight(f"Answer {i+1}: {states[i]['answer']}")

[2025-02-26 23:40:41 TP0] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 13, cache hit rate: 35.74%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-02-26 23:40:41] INFO:     127.0.0.1:53320 - "POST /generate HTTP/1.1" 200 OK


  0%|          | 0/3 [00:00<?, ?it/s]

 33%|███▎      | 1/3 [00:00<00:00,  7.04it/s]

 67%|██████▋   | 2/3 [00:00<00:00, 11.15it/s]

[2025-02-26 23:40:41 TP0] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 17, cache hit rate: 36.64%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-02-26 23:40:41 TP0] Prefill batch. #new-seq: 1, #new-token: 11, #cached-token: 17, cache hit rate: 37.40%, token usage: 0.00, #running-req: 1, #queue-req: 0
[2025-02-26 23:40:41 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 19, cache hit rate: 38.42%, token usage: 0.00, #running-req: 2, #queue-req: 0
[2025-02-26 23:40:41] INFO:     127.0.0.1:53332 - "POST /generate HTTP/1.1" 200 OK
[2025-02-26 23:40:41] INFO:     127.0.0.1:53356 - "POST /generate HTTP/1.1" 200 OK


100%|██████████| 3/3 [00:00<00:00, 18.12it/s]

[2025-02-26 23:40:41] INFO:     127.0.0.1:53348 - "POST /generate HTTP/1.1" 200 OK


## Streaming 

Use `stream` to stream the output to the user.

In [11]:
@function
def text_qa(s, question):
    s += user(question)
    s += assistant(gen("answer", stop="\n"))


state = text_qa.run(
    question="What is the capital of France?", temperature=0.1, stream=True
)

for out in state.text_iter():
    print(out, end="", flush=True)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
What is the capital of France?<|im_end|>
<|im_start|>assistant


[2025-02-26 23:40:41] INFO:     127.0.0.1:53360 - "POST /generate HTTP/1.1" 200 OK
[2025-02-26 23:40:41 TP0] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 25, cache hit rate: 40.02%, token usage: 0.00, #running-req: 0, #queue-req: 0


The

 capital

 of

[2025-02-26 23:40:41 TP0] Decode batch. #running-req: 1, #token: 30, token usage: 0.00, gen throughput (token/s): 90.47, #queue-req: 0
 France

 is

 Paris

.

<|im_end|>


## Complex Prompts

You may use `{system|user|assistant}_{begin|end}` to define complex prompts.

In [12]:
@function
def chat_example(s):
    s += system("You are a helpful assistant.")
    # Same as: s += s.system("You are a helpful assistant.")

    with s.user():
        s += "Question: What is the capital of France?"

    s += assistant_begin()
    s += "Answer: " + gen("answer", max_tokens=100, stop="\n")
    s += assistant_end()


state = chat_example()
print_highlight(state["answer"])

[2025-02-26 23:40:41 TP0] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 14, cache hit rate: 40.19%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-02-26 23:40:41] INFO:     127.0.0.1:53374 - "POST /generate HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

## Multi-modal Generation

You may use SGLang frontend language to define multi-modal prompts.
See [here](https://docs.sglang.ai/references/supported_models.html) for supported models.

In [14]:
server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path Qwen/Qwen2.5-VL-7B-Instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")
print(f"Server started on http://localhost:{port}")

[2025-02-26 23:40:54] server_args=ServerArgs(model_path='Qwen/Qwen2.5-VL-7B-Instruct', tokenizer_path='Qwen/Qwen2.5-VL-7B-Instruct', tokenizer_mode='auto', load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization_param_path=None, quantization=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen2.5-VL-7B-Instruct', chat_template=None, is_embedding=False, revision=None, skip_tokenizer_init=False, host='0.0.0.0', port=37486, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, cpu_offload_gb=0, prefill_only_one_req=False, tp_size=1, stream_interval=1, stream_output=False, random_seed=18012216, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, log_level='info', log_level_http=None, log_requests=False, show_time_cost=False, enable_metrics=Fal

[2025-02-26 23:41:13 TP0] Overlap scheduler is disabled for multimodal models.


[2025-02-26 23:41:13 TP0] Automatically reduce --mem-fraction-static to 0.836 because this is a multimodal model.
[2025-02-26 23:41:13 TP0] Init torch distributed begin.
[2025-02-26 23:41:13 TP0] Load weight begin. avail mem=59.67 GB


[2025-02-26 23:41:14 TP0] The following error message 'operation scheduled before its operands' can be ignored.


[2025-02-26 23:41:15 TP0] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/5 [00:00<?, ?it/s]


...

[2025-02-26 23:41:20 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=128000
[2025-02-26 23:41:20] INFO:     Started server process [1863356]
[2025-02-26 23:41:20] INFO:     Waiting for application startup.
[2025-02-26 23:41:20] INFO:     Application startup complete.
[2025-02-26 23:41:20] INFO:     Uvicorn running on http://0.0.0.0:37486 (Press CTRL+C to quit)


[2025-02-26 23:41:20] INFO:     127.0.0.1:46220 - "GET /v1/models HTTP/1.1" 200 OK


[2025-02-26 23:41:21] INFO:     127.0.0.1:57668 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-02-26 23:41:21 TP0] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-02-26 23:41:24] INFO:     127.0.0.1:57684 - "POST /generate HTTP/1.1" 200 OK
[2025-02-26 23:41:24] The server is fired up and ready to roll!


Server started on http://localhost:37486


In [15]:
set_default_backend(RuntimeEndpoint(f"http://localhost:{port}"))

[2025-02-26 23:41:25] INFO:     127.0.0.1:57690 - "GET /get_model_info HTTP/1.1" 200 OK


Ask a question about an image.

In [16]:
@function
def image_qa(s, image_file, question):
    s += user(image(image_file) + question)
    s += assistant(gen("answer", max_tokens=256))


image_url = "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true"
image_bytes, _ = load_image(image_url)
state = image_qa(image_bytes, "What is in the image?")
print_highlight(state["answer"])

[2025-02-26 23:41:34 TP0] Prefill batch. #new-seq: 1, #new-token: 307, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-02-26 23:41:38 TP0] Decode batch. #running-req: 1, #token: 340, token usage: 0.02, gen throughput (token/s): 2.20, #queue-req: 0


[2025-02-26 23:41:39 TP0] Decode batch. #running-req: 1, #token: 380, token usage: 0.02, gen throughput (token/s): 59.78, #queue-req: 0


[2025-02-26 23:41:39 TP0] Decode batch. #running-req: 1, #token: 420, token usage: 0.02, gen throughput (token/s): 59.96, #queue-req: 0


[2025-02-26 23:41:40 TP0] Decode batch. #running-req: 1, #token: 460, token usage: 0.02, gen throughput (token/s): 59.70, #queue-req: 0
[2025-02-26 23:41:40] INFO:     127.0.0.1:59194 - "POST /generate HTTP/1.1" 200 OK


In [17]:
terminate_process(server_process)